In [1]:
#importer les bibliothèques nécessaire
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import pandas as pd

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:

def nettoyer_texte(texte):
    """
    Nettoie le texte en supprimant mentions, hashtags, URLs, ponctuation, emojis
    """
    if not isinstance(texte, str):
        return ""
    
    # Supprimer les mentions @
    texte = re.sub(r"@\w+", "", texte)
    # Supprimer les hashtags #
    texte = re.sub(r"#\w+", "", texte)
    # Supprimer les URLs
    texte = re.sub(r"http\S+|www\.\S+", "", texte)
    # Supprimer la ponctuation et caractères spéciaux (garder lettres et accents)
    texte = re.sub(r"[^a-zàâäéèêëîïôöùûüç\s]", "", texte, flags=re.IGNORECASE)
    # Convertir en minuscules
    texte = texte.lower()
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte)
    
    return texte.strip()

# Initialiser les ressources françaises
french_stopwords = set(stopwords.words('french'))
stemmer = SnowballStemmer("french")

def preprocess_text(texte):
    """
    Prétraitement complet du texte
    """
    # Nettoyage de base
    texte_nettoye = nettoyer_texte(texte)
    
    # Tokenisation
    tokens = word_tokenize(texte_nettoye, language='french')
    
    # Suppression des stopwords et stemming
    tokens_filtres = [
        stemmer.stem(token) 
        for token in tokens 
        if token not in french_stopwords and len(token) > 2
    ]
    
    return tokens_filtres

def fusionner_et_pretraiter():
    # 1. Charger les deux fichiers
    df_uvbf = pd.read_csv('data_uvbf.csv', encoding='utf-8')
    df_facebook = pd.read_csv('facebook_posts.csv', encoding='utf-8')
    
    # 2. Ajouter une colonne pour identifier la source
    df_uvbf['source'] = 'sondage'
    df_facebook['source'] = 'facebook'
    
    # 3. Fusionner les deux dataframes
    df_fusion = pd.concat([df_uvbf, df_facebook], ignore_index=True, sort=False)
    
    print("\nFusion terminée - Dimensions du DataFrame fusionné:", df_fusion.shape)
    
    # 4. Identifier toutes les colonnes de texte
    colonnes_texte = []
    for colonne in df_fusion.columns:
        if df_fusion[colonne].dtype == 'object' and colonne not in ['source', 'Timestamp', 'profile_url', 'extracted_at']:
            colonnes_texte.append(colonne)

    
    print(f"\nColonnes de texte identifiées: {colonnes_texte}")
    
    # 5. Appliquer le prétraitement sur chaque colonne de texte
    for colonne in colonnes_texte:
        print(f"Traitement de la colonne: {colonne}")
        
        # Créer une nouvelle colonne pour les tokens prétraités
        nouvelle_colonne = f"{colonne}_pretraite"
        df_fusion[nouvelle_colonne] = df_fusion[colonne].apply(preprocess_text)
        
        # Afficher un échantillon des résultats
        samples = df_fusion[df_fusion[colonne].notna()].head(2)
        for i, (orig, pretrait) in enumerate(zip(samples[colonne], samples[nouvelle_colonne])):
            if pd.notna(orig):
                print(f"  Exemple {i+1}:")
                print(f"    Original: {str(orig)[:80]}...")
                print(f"    Prétraité: {pretrait}")

    df_fusion['texte_complet_pretraite'] = df_fusion[[f"{col}_pretraite" for col in colonnes_texte]].apply(
        lambda row: [token for col_pretraite in row.index for token in row[col_pretraite] if isinstance(row[col_pretraite], list)], 
        axis=1
    )

    
    # 7. Statistiques sur le prétraitement
    print(f"Nombre total d'observations: {len(df_fusion)}")

    for colonne in colonnes_texte:
        nouvelle_colonne = f"{colonne}_pretraite"
        nb_tokens = df_fusion[nouvelle_colonne].apply(lambda x: len(x) if isinstance(x, list) else 0).sum()
        print(f"Nombre total de tokens dans {colonne}: {nb_tokens}")
           
    
    # 6. Sauvegarder le fichier pretraiter
    df_fusion.to_csv('donnees_pretraitees.csv', index=False, encoding='utf-8')
    return df_fusion

# Exécuter la fonction
if __name__ == "__main__":
    print("Début du prétraitement des données...")
    df_resultat = fusionner_et_pretraiter()
    print("Prétraitement terminer")
    print(f"Données traitées : {len(df_resultat)} lignes ")
    


Début du prétraitement des données...

Fusion terminée - Dimensions du DataFrame fusionné: (172, 9)

Colonnes de texte identifiées: ["Qu'est-ce que vous appréciez le plus à l'UV-BF ?", "Quelles sont les principales difficultés ou aspects négatifs que vous rencontrez à l'UV-BF ?", 'profile_name', 'post_text', 'comments']
Traitement de la colonne: Qu'est-ce que vous appréciez le plus à l'UV-BF ?
  Exemple 1:
    Original: Les cours sont intéressants mais parfois difficiles à suivre à cause des coupure...
    Prétraité: ['cour', 'intéress', 'parfois', 'difficil', 'suivr', 'caus', 'coupur', 'connexion']
  Exemple 2:
    Original: Oui je pense que il ya des failles ! La connexion n'est pas stable, et en plus l...
    Prétraité: ['oui', 'pens', 'faill', 'connexion', 'nest', 'stabl', 'plus', 'qualit', 'cour', 'nest', 'trop']
Traitement de la colonne: Quelles sont les principales difficultés ou aspects négatifs que vous rencontrez à l'UV-BF ?
  Exemple 1:
    Original: Les études à l'Uv, c'est